## Model Valuation

We have several model types that we have been creating over the course of the semester. Some of the models (e.g. regression_models.ipynb and election_betting.ipynb) make one-step predictions. By this, I mean that they take in the most recent polling/betting data and then try to predict what the polls will be tomorrow. Typically this prediction is dependent on the poll company itself. In this way, we can really describe our models as naive markov chains. The next state (poll result j) is a function of the current polling data and the "random" betting data. 

We can evaluate these models by simply checking the polls that were actually released that date and finding a mean squared error. 

In [1]:
#Let's start with the betting data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from realclearpolling import *

#Read in the data
df = pd.read_csv('Datasets/betting_predictions.csv')
df.set_index('Pollster', inplace=True)
df.head()

,Biden Prediction: 2024-03-19,Trump Prediction: 2024-03-19,Biden Prediction: 2024-03-21,Trump Prediction: 2024-03-21,Biden Prediction: 2024-03-22,Trump Prediction: 2024-03-22,Biden Prediction: 2024-03-23,Trump Prediction: 2024-03-23,Biden Prediction: 2024-03-24,Trump Prediction: 2024-03-24,Biden Prediction: 2024-03-26,Trump Prediction: 2024-03-26,Biden Prediction: 2024-03-27,Trump Prediction: 2024-03-27,Biden Prediction: 2024-03-28,Trump Prediction: 2024-03-28,Biden Prediction: 2024-03-29,Trump Prediction: 2024-03-29,Biden Prediction: 2024-04-08,Trump Prediction: 2024-04-08
Pollster,,,,,,,,,,,,,,,,,,,,
pollster_CBS News,49.609570,49.113020,49.185043,48.829216,49.185043,48.829216,49.185043,48.829216,49.185043,48.829216,49.226624,49.201122,48.760284,49.148335,48.760284,49.148335,48.957010,48.524178,49.243904,49.024685
pollster_CNBC,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,44.865920,45.311665,44.865920,45.311665,44.935720,44.717525,45.117850,44.405327
pollster_CNN,45.929600,47.225773,46.110542,47.152060,46.110542,47.152060,46.110542,47.152060,46.110542,47.152060,45.733463,47.166122,45.737206,47.232560,45.737206,47.232560,45.722393,46.976315,45.744347,47.023590
pollster_Daily Kos/Civiqs,45.101093,43.993378,45.241467,44.505825,45.241467,44.505825,45.241467,44.505825,45.241467,44.505825,45.237000,44.781246,44.865920,44.774180,44.865920,44.774180,44.935720,43.727890,44.988018,44.223034
pollster_Data for Progress (D)**,45.801079,43.447899,46.574776,43.742943,46.574776,43.742943,46.574776,43.742943,46.574776,43.742943,46.519714,43.901270,46.611420,44.686535,46.611420,44.686535,46.450077,43.692757,45.939540,43.344505


In [2]:
#For each date, we want to see which polls were released on rcp
rcp = clean_data(get_poll_data())
rcp.head()

,pollster,date,sample,moe,Trump (R),Biden (D),spread,Difference,Type of Voter,Sample Size,End Date,Poll Month,Year,Days Since 01-01-23
0,Emerson,4/2 - 4/3,1438 RV,2.5,46.0,45.0,Trump+1,1.0,RV,1438,2024-04-03,4,2024,458
1,Rasmussen Reports,3/31 - 4/2,1099 LV,3.0,49.0,41.0,Trump+8,8.0,LV,1099,2024-04-02,4,2024,457
2,Morning Consult,3/29 - 3/31,6018 RV,1.0,42.0,44.0,Biden+2,-2.0,RV,6018,2024-03-31,3,2024,455
3,Data for Progress (D)**,3/27 - 3/29,1200 LV,3.0,46.0,47.0,Biden+1,-1.0,LV,1200,2024-03-29,3,2024,453
4,NPR/PBS/Marist,3/25 - 3/28,1199 RV,3.7,48.0,50.0,Biden+2,-2.0,RV,1199,2024-03-28,3,2024,452


In [3]:
#Betting dates
dates = []
for col in df.columns:
    if ':' in col:
        ind = col.index(':')+2
        date = col[ind:]
        #Convert to datetime
        date = datetime.strptime(date, '%Y-%m-%d')
        dates.append(date)
print(dates)

[datetime.datetime(2024, 3, 19, 0, 0), datetime.datetime(2024, 3, 19, 0, 0), datetime.datetime(2024, 3, 21, 0, 0), datetime.datetime(2024, 3, 21, 0, 0), datetime.datetime(2024, 3, 22, 0, 0), datetime.datetime(2024, 3, 22, 0, 0), datetime.datetime(2024, 3, 23, 0, 0), datetime.datetime(2024, 3, 23, 0, 0), datetime.datetime(2024, 3, 24, 0, 0), datetime.datetime(2024, 3, 24, 0, 0), datetime.datetime(2024, 3, 26, 0, 0), datetime.datetime(2024, 3, 26, 0, 0), datetime.datetime(2024, 3, 27, 0, 0), datetime.datetime(2024, 3, 27, 0, 0), datetime.datetime(2024, 3, 28, 0, 0), datetime.datetime(2024, 3, 28, 0, 0), datetime.datetime(2024, 3, 29, 0, 0), datetime.datetime(2024, 3, 29, 0, 0), datetime.datetime(2024, 4, 8, 0, 0), datetime.datetime(2024, 4, 8, 0, 0)]


In [4]:
#Get the polls for each date
rcp["End Date"] = pd.to_datetime(rcp["End Date"])
subset = rcp[rcp["End Date"].isin(dates)]
shared_dates = subset["End Date"]
shared_polls = subset["pollster"]

In [5]:
#Once we collect enough data, we can plot the differences
#For each poll on each date, get the value of Trump and the Value of Biden
#Then compare to the predicted poll from the betting data
#Make a new dataframe with the poll, the date, the predicted value, the actual value, and the difference
#Then plot the difference

biden_predictions = []
trump_predictions = []
biden_actual = []
trump_actual = []

for i in range(len(shared_dates)):
    date = shared_dates[i]
    poll = shared_polls[i]
    #Get the predicted value
    date_string = date.strftime('%Y-%m-%d')
    poll_name = "pollster_" + poll
    biden_pred = df["Biden Prediction: " + date_string].loc[poll_name]
    biden_predictions.append(biden_pred)
    trump_pred = df["Trump Prediction: " + date_string].loc[poll_name]
    trump_predictions.append(trump_pred)
    #Get the actual value
    biden_act = rcp[(rcp["End Date"] == date) & (rcp["pollster"] == poll)]["Biden (D)"].values.tolist()[0]
    biden_actual.append(biden_act)
    trump_act = subset[subset["pollster"] == poll]["Trump (R)"].values.tolist()[0]
    trump_actual.append(trump_act)

KeyError: 0

In [ ]:
#Make a new dataframe
data = {"Poll": shared_polls, "Date": shared_dates, "Biden Prediction": biden_predictions, "Trump Prediction": trump_predictions, "Biden Actual": biden_actual, "Trump Actual": trump_actual}
betting_validation = pd.DataFrame(data)
betting_validation["Biden Difference"] = betting_validation["Biden Prediction"] - betting_validation["Biden Actual"]
betting_validation["Trump Difference"] = betting_validation["Trump Prediction"] - betting_validation["Trump Actual"]
betting_validation.head()

,Poll,Date,Biden Prediction,Trump Prediction,Biden Actual,Trump Actual,Biden Difference,Trump Difference
0,CNBC,2024-03-19,-1.000000,-1.000000,45.0,46.0,-46.000000,-47.000000
1,Morning Consult,2024-03-24,43.778572,42.487514,44.0,43.0,-0.221428,-0.512486
2,Harvard-Harris,2024-03-21,42.717533,45.264015,49.0,51.0,-6.282467,-5.735985


In [ ]:
#Repeat with the most larger polling data
#We have been saving this data in a different format, so we will need to handle it differently
linear_regression_model = pd.ExcelFile("Datasets/Predictions.xlsx")

# Read each sheet into a DataFrame
dfs = [pd.read_excel(linear_regression_model, sheet_name=sheet_name) for sheet_name in linear_regression_model.sheet_names]
merged_df = pd.merge(dfs[0], dfs[1], on='Unnamed: 0')
merged_df
# # Loop through the remaining DataFrames and merge them with the merged DataFrame
# for df in dfs[2:]:
#     merged_df = pd.merge(merged_df, df, on='Unnamed: 0', suffixes=('_left', '_right'))
# merged_df.set_index('Unnamed: 0', inplace=True)

,Unnamed: 0,Predicted Difference_x,Predicted Difference_y
0,pollster_CBS News,2.514775,2.540727
1,pollster_CNN,3.509744,3.535495
2,pollster_Daily Kos/Civiqs,0.105638,0.115123
3,pollster_Data for Progress (D)**,-1.733390,-1.717426
4,pollster_Economist/YouGov,0.454308,0.491706
5,pollster_Emerson,2.234276,2.279126
6,pollster_FOX News,1.856966,1.888643
7,pollster_Federalist/Susquehanna,-1.646830,-1.587132
8,pollster_Grinnell/Selzer,0.638857,0.669804
9,pollster_Harvard-Harris,5.365957,5.414900
